In [1]:
#import libraries
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import time

In [2]:
PATH ="C:/Users/DELL/chromedriver.exe"
options = webdriver.ChromeOptions()

#options.add_argument('--headless')
# Adding arguments to the ChromeOptions object
options.add_argument('--no-sandbox')# Disables the use of the sandbox for added security
options.add_argument('--disable-dev-shm-usage')# Disables the use of /dev/shm
options.add_argument()

# Defining the URLs to be visited
web = "https://www.rightmove.co.uk/property-for-sale/London.html"
web1 = "https://www.rightmove.co.uk/property-to-rent/London.html"

# Creating a ChromeDriver instance with the specified service and options
driver = webdriver.Chrome(service = Service(PATH))

# Opening the specified URL in the Chrome browser
driver.get(web)

In [3]:
# Initializing lists to store scraped data
no_of_bedrooms = []
no_of_bathrooms = []
descr = []
prop_type = []
addr = []
agent = []
l_source = []
l_url = []

# Infinite loop to scrape data from multiple pages
while True:
    # Scroll to the bottom of the page to load more listings
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5) # Wait for 5 seconds to load the new content
    
    # Extract the HTML source code of the page and parse it with BeautifulSoup
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # Extracting the number of bedrooms for each property
    for i in soup.find_all('div', class_ = 'propertyCard-content'):
        j = i.find('span', class_ = 'no-svg-bed-icon bed-icon seperator')
        if j != None:
            no_of_bedrooms.append(j.next_sibling.text)
        else:
            no_of_bedrooms.append(None)
    
    # Extracting the number of bathrooms for each property
    for i in soup.find_all('div', class_ = 'propertyCard-content'):
        j = i.find('span', class_ = 'no-svg-bathroom-icon bathroom-icon seperator')
        if j != None:
            no_of_bathrooms.append(j.next_sibling.text)
        else:
            no_of_bathrooms.append(None)
    
    # Extracting the description for each property
    for i in soup.find_all('div', class_ = 'propertyCard-content'):
        j = i.find('span', attrs = {'itemprop': 'description'})
        if j != None:
            descr.append(j.text)
        else:
            descr.append(None)
    
    # Extracting the property type for each property
    for i in soup.find_all('div', class_ = 'propertyCard-content'):
        j = i.find('span', class_ = 'text')
        if j != None:
            prop_type.append(j.text)
        else:
            prop_type.append(None)
            
    # Extracting the location for each property
    for i in soup.find_all('div', class_ = 'propertyCard-content'):
        j = i.find('div', class_ = 'propertyCard-branchSummary property-card-updates').text
        if j != None:
            addr.append(j)
        else:
            addr.append(None)
    
    # Extracting the agent for each property
    for i in soup.find_all('div', class_ = 'propertyCard-content'):
        j = i.find('div', class_ = 'propertyCard-branchSummary property-card-updates')
        j_text = j.text
        if j != None:
            if 'by' in j_text:
                n = j_text.index('by')
                agent.append(j_text[n:])
        else:
            agent.append(None)
    
    # Extracting the listing source for each property
    for i in soup.find_all('div', class_ = 'propertyCard-content'):
        j = i.find('a', class_ = 'propertyCard-link property-card-updates')
        if j != None:
            l_source.append(re.search(r"[a-zA-Z]+\W+[a-zA-Z.]+",web).group() + j['href'])
        else:
            l_source.append(None)
            
    # Extracting the listing URL for each property
    for i in soup.find_all('div', class_ = 'propertyCard-content'):
        j = i.find('a', class_ = 'propertyCard-branchLogo-link')
        if j != None:
            l_url.append(re.search(r"[a-zA-Z]+\W+[a-zA-Z.]+", web).group() +j['href'])
        else:
            l_url.append(None)
    
    # Finding the next page button and clicking on it
    next_page = driver.find_element(By.CSS_SELECTOR,"button.pagination-button.pagination-direction.pagination-direction--next")
    
    
    if  driver.execute_script(" return arguments[0].disabled;",next_page) == True:
        # If the next page button is disabled, break out of the loop
        break
    # If the next page button is not disabled, click on it to load the next page
    else:
        driver.execute_script("arguments[0].click();",next_page)

    
    # Add some delay to avoid overloading the server
    time.sleep(5)

# Creating a dictionary to store the scraped data
dic = {'Transaction Type': ['for sale'] * len(l_url), 'Bedrooms': no_of_bedrooms, 'Bathrooms': no_of_bathrooms,
      'Description': descr, 'Property Type': prop_type, 'Location': addr, 'Agent': agent, 'Listing Source': l_source,
      'Listing URL': l_url, 'Furnished': [None]*len(l_url)}

# Creating a DataFrame using the dictionary
df_buy = pd.DataFrame(dic)

In [5]:
#Save our dataframe to csv file
df_buy.to_csv('For Sale Scraping.csv', index = False)

In [6]:
df_buy

,Transaction Type,Bedrooms,Bathrooms,Description,Property Type,Location,Agent,Listing Source,Listing URL,Furnished
0,for sale,3,1,Huge potential to extend,Semi-Detached,"Added on 09/01/2023 by haart, Walthamstow","by haart, Walthamstow",/properties/130506473#/?channel=RES_BUY,/estate-agents/agent/haart/Walthamstow-11600.html,None
1,for sale,5,5,An exceptional exclusive five bedroom apartmen...,Apartment,"Added on 22/12/2022 by The Cloister, London","by The Cloister, London",/properties/130177625#/?channel=RES_BUY,/estate-agents/agent/The-Cloister/London-51305...,None
2,for sale,12,18,This stunning 12 bedroom (including 3 staff be...,House,Added on 31/12/2022 by Luxury Living Homes Int...,"by Luxury Living Homes International, London",/properties/130306529#/?channel=RES_BUY,/estate-agents/agent/Luxury-Living-Homes-Inter...,None
3,for sale,7,9,Ref. LOB0798 - Set behind a gated Belgravia dr...,House,"Added on 21/11/2022 by Beauchamp Estates Ltd, ...","by Beauchamp Estates Ltd, Mayfair - Resale",/properties/129302918#/?channel=RES_BUY,/estate-agents/agent/Beauchamp-Estates-Ltd/May...,None
4,for sale,7,5,"A rare opportunity to own this unique, contemp...",Detached,"Added on 20/02/2023 by Savills, Kensington","by Savills, Kensington",/properties/131846453#/?channel=RES_BUY,/estate-agents/agent/Savills/Kensington-48934....,None
...,...,...,...,...,...,...,...,...,...,...
1045,for sale,2,2,Apartment A204 is located on the second floor ...,Apartment,"Added on 09/11/2022 by Winkworth, Paddington &...","by Winkworth, Paddington & Bayswater",/properties/128908406#/?channel=RES_BUY,/estate-agents/agent/Winkworth/Paddington-and-...,None
1046,for sale,3,3,A rare opportunity to purchase an outstanding ...,Flat,"Added on 02/05/2023 by Knight Frank, Richmond","by Knight Frank, Richmond",/properties/134337323#/?channel=RES_BUY,/estate-agents/agent/Knight-Frank/Richmond-288...,None
1047,for sale,6,4,jdm Estate Agents are delighted to present thi...,Detached,"Added on 26/01/2023 by jdm, Locksbottom","by jdm, Locksbottom",/properties/131118500#/?channel=RES_BUY,/estate-agents/agent/jdm/Locksbottom-7408.html,None
1048,for sale,6,6,Prime Collection are delighted to offer this m...,Terraced,"Added on 16/05/2023 by Prime Collection, London","by Prime Collection, London",/properties/134883356#/?channel=RES_BUY,/estate-agents/agent/Prime-Collection/London-2...,None
